## Прогнозирование цен на автомобили

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedShuffleSplit 
from sklearn.model_selection import GridSearchCV,  RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.linear_model import  Lars, OrthogonalMatchingPursuit, SGDRegressor, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification

from sklearn.feature_selection  import RFE
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import VarianceThreshold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.inspection import permutation_importance

from scipy.stats import spearmanr
from scipy.spatial.distance import squareform
from scipy.cluster import hierarchy
from collections import defaultdict
from sklearn.preprocessing import PowerTransformer

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor

In [4]:
path_train = 'train.csv' # содержит только имя файла, без имен папок !!!

In [5]:
df = pd.read_csv(path_train)

In [6]:
display(df.head())
display(df.info())
display(df.describe())

,company,model,year,condition,mileage(km),fuel,volume(cm3),color,transmission,drive_unit,vehicle_size_class,priceUSD
0,renault,laguna,2000,with mileage,230000.0,petrol,1800.0,blue,mechanics,front-wheel drive,D,2300
1,chrysler,pacifica,2005,with mileage,310000.0,petrol,3500.0,black,auto,front-wheel drive,NaN,6300
2,mercedes-benz,c-klass,2010,with mileage,115000.0,petrol,1800.0,gray,auto,rear drive,D,11200
3,hyundai,accent,2017,with mileage,45500.0,petrol,1400.0,gray,auto,front-wheel drive,B,10886
4,renault,kadjar,2016,with mileage,139000.0,diesel,1500.0,red,auto,front-wheel drive,J,15300


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38005 entries, 0 to 38004
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   company             38005 non-null  object 
 1   model               38005 non-null  object 
 2   year                38005 non-null  int64  
 3   condition           38005 non-null  object 
 4   mileage(km)         38005 non-null  float64
 5   fuel                38005 non-null  object 
 6   volume(cm3)         38005 non-null  float64
 7   color               38005 non-null  object 
 8   transmission        38005 non-null  object 
 9   drive_unit          38005 non-null  object 
 10  vehicle_size_class  34434 non-null  object 
 11  priceUSD            38005 non-null  int64  
dtypes: float64(2), int64(2), object(8)
memory usage: 3.5+ MB


None

,year,mileage(km),volume(cm3),priceUSD
count,38005.000000,3.800500e+04,38005.000000,38005.000000
mean,2003.468833,2.456028e+05,2103.036732,7425.598027
std,8.118155,3.285505e+05,925.249883,8404.015035
min,1910.000000,0.000000e+00,500.000000,48.000000
25%,1998.000000,1.380000e+05,1600.000000,2377.000000
50%,2004.000000,2.300000e+05,1997.000000,5390.000000
75%,2010.000000,3.100000e+05,2300.000000,9850.000000
max,2019.000000,9.999999e+06,20000.000000,235235.000000


In [7]:
#df[df['vehicle_size_class'].isna()]

In [8]:
df.drop(columns='color', inplace=True) #очень сильно ухудшает метрику

In [9]:
df.drop_duplicates(inplace=True)
# удалить пропущенные данные
display(df.isna().sum())

# vehicle_size_class имеет много пропущенных данных 10%
# в инструкции еще и написано, что могут быть ошибки
df.dropna(inplace=True)
display(df.isna().sum())

company                  0
model                    0
year                     0
condition                0
mileage(km)              0
fuel                     0
volume(cm3)              0
transmission             0
drive_unit               0
vehicle_size_class    3566
priceUSD                 0
dtype: int64

company               0
model                 0
year                  0
condition             0
mileage(km)           0
fuel                  0
volume(cm3)           0
transmission          0
drive_unit            0
vehicle_size_class    0
priceUSD              0
dtype: int64

In [10]:
df['age'] = 2020 - df['year']
df.drop(columns='year', inplace=True)
df

,company,model,condition,mileage(km),fuel,volume(cm3),transmission,drive_unit,vehicle_size_class,priceUSD,age
0,renault,laguna,with mileage,230000.0,petrol,1800.0,mechanics,front-wheel drive,D,2300,20
2,mercedes-benz,c-klass,with mileage,115000.0,petrol,1800.0,auto,rear drive,D,11200,10
3,hyundai,accent,with mileage,45500.0,petrol,1400.0,auto,front-wheel drive,B,10886,3
4,renault,kadjar,with mileage,139000.0,diesel,1500.0,auto,front-wheel drive,J,15300,4
5,ford,escort,with mileage,325000.0,petrol,1360.0,mechanics,front-wheel drive,C,575,28
...,...,...,...,...,...,...,...,...,...,...,...
37999,bmw,x5,with mileage,310000.0,diesel,3500.0,auto,part-time four-wheel drive,J,16000,12
38000,renault,laguna,with mileage,314000.0,petrol,1600.0,mechanics,front-wheel drive,D,1999,21
38001,opel,insignia,with mileage,230000.0,petrol,1600.0,mechanics,front-wheel drive,D,7606,10
38002,kia,optima,with mileage,115000.0,petrol,2400.0,mechanics,front-wheel drive,D,11850,8


In [11]:
df[['vehicle_size_class', 'priceUSD']].groupby('vehicle_size_class').mean().sort_values(by='priceUSD')

,priceUSD
vehicle_size_class,
A,4233.085809
B,4271.096049
C,4843.015581
D,5374.956081
E,6235.373678
M,6942.696111
F,10926.523397
J,14438.935971
S,17236.432122


In [12]:
categorical_features_1 = ['fuel', 'transmission'] #'color' #'condition','drive_unit'
categorical_features_2 = ['company', 'model']
numerical_features = ['age', 'mileage(km)',  'volume(cm3)']

In [13]:
y = np.array(df['priceUSD'])
X = df.drop(columns=['priceUSD'])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

In [14]:
# numerical = Pipeline(
#     steps=[
#     ('missing_num', IterativeImputer(missing_values=0, max_iter=20)),
#     ('scaler', MinMaxScaler())]
# )

# class_map = [{
#     'col': 'vehicle_size_class',
#     'mapping': {'S':8, 'J':7, 'F':6, 'M':5, 
#                'E':4, 'D':3, 'C':2, 'B':1, 'A':0}
#     }]

# class_transformer = Pipeline(
#     steps=[
#     ('missing_num', SimpleImputer(strategy = 'most_frequent')),
#     ('ce',ce.OrdinalEncoder(mapping=class_map)),
#     ('scaler', MinMaxScaler())
#     ])    
    
# data_transformer = ColumnTransformer([
#         ("numerical", numerical, numerical_features),
#         # убрать быстую разряженную и затем добавить тип pandas
#         ("cat_1", OneHotEncoder(sparse_output=False).set_output(transform="pandas"), categorical_features_1),
#         ("cat_2", OrdinalEncoder(), categorical_features_2),
#         ("ordinal_map", class_transformer, ['vehicle_size_class'])
#         ]).set_output(transform='pandas')

# # Создание конвейера препроцессора, который сначала преобразует данные.
# preprocessor = Pipeline(steps=[("data_transformer", data_transformer)])

# classifier_pipline = Pipeline(
#     steps=[('preprocessor', preprocessor),
#            ('knn', KNeighborsRegressor())])


In [15]:
numerical = Pipeline(
    steps=[
    ('missing_num', IterativeImputer(missing_values=0, max_iter=20)),
    ('scaler', StandardScaler())]
)

class_map = [{
    'col': 'vehicle_size_class',
    'mapping': {'S':8, 'J':7, 'F':6, 'M':5, 
               'E':4, 'D':3, 'C':2, 'B':1, 'A':0}
    }]

class_transformer = Pipeline(
    steps=[
    ('missing_num', SimpleImputer(strategy = 'most_frequent')),
    ('ce',ce.OrdinalEncoder(mapping=class_map)),
    ('scaler', StandardScaler())
    ])  

drive_unit_map = [{
    'col': 'drive_unit',
    'mapping': {'front-wheel drive':0, 'rear drive':1, 'part-time four-wheel drive':2, 'all-wheel drive':3}
    }]

drive_unit_transformer = Pipeline(
    steps=[
    ('missing_num', SimpleImputer(strategy = 'most_frequent')),
    ('ce',ce.OrdinalEncoder(mapping=drive_unit_map)),
    ('scaler', StandardScaler())
    ])  

condition_map = [{
    'col': 'condition',
    'mapping': {'for parts':0, 'with damage':1, 'with mileage':2}
    }]

condition_transformer = Pipeline(
    steps=[
    ('missing_num', SimpleImputer(strategy = 'most_frequent')),
    ('ce',ce.OrdinalEncoder(mapping=condition_map)),
    ('scaler', StandardScaler())
    ])  

company = Pipeline(
    steps=[('enc', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value = 51)), #51,91
    ('scaler', StandardScaler())]
)

model = Pipeline(
    steps=[('enc', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value = 454)), #454,972
    ('scaler', StandardScaler())]
)
    
    
# company = Pipeline(
#     steps=[('enc1', LabelEncoder()),
#     ('scaler', MinMaxScaler())]
# )

# model = Pipeline(
#     steps=[('enc2', LabelEncoder()),
#     ('scaler', MinMaxScaler())]
# )
    
data_transformer = ColumnTransformer([
        ("numerical", numerical, numerical_features),
        # убрать быстую разряженную и затем добавить тип pandas
        ("cat_1", OneHotEncoder(sparse_output=False).set_output(transform="pandas"), categorical_features_1),
        ("model", model, ['model']),
        ("company", company, ['company']),
        ("ordinal_map", class_transformer, ['vehicle_size_class']),
        ("ordinal_map_2", drive_unit_transformer, ['drive_unit']),
        ("ordinal_map_3", condition_transformer, ['condition'])
        ]).set_output(transform='pandas')

# Создание конвейера препроцессора, который сначала преобразует данные.
preprocessor = Pipeline(steps=[("data_transformer", data_transformer)])

model_pipline = Pipeline(
    steps=[('preprocessor', preprocessor),
           ('knn', KNeighborsRegressor())])


In [16]:
df['model'].nunique()

454

In [17]:
df['company'].nunique()

51

In [18]:
 
# scaling = StandardScaler().set_output(transform='pandas')
# df = scaling.fit_transform(df)

In [19]:
# Определим параметры для перебора по решетке
p = np.arange(1,5)
n_neighbors = np.arange(1,15)
scaling = [ MinMaxScaler(), StandardScaler(),RobustScaler() ]

# Подбор параметров для модели
param_grid =[
    {'knn': [KNeighborsRegressor()],
         'knn__n_neighbors': n_neighbors,
         'knn__p': p
     }]

result = GridSearchCV(model_pipline, param_grid, n_jobs=-1, cv=10, scoring = 'neg_mean_absolute_percentage_error') 

result.fit(X_train, y_train)


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        Pipeline(steps=[('data_transformer',
                                                         ColumnTransformer(transformers=[('numerical',
                                                                                          Pipeline(steps=[('missing_num',
                                                                                                           IterativeImputer(max_iter=20,
                                                                                                                            missing_values=0)),
                                                                                                          ('scaler',
                                                                                                           StandardScaler())]),
                                                                                          ['age',
                                                                                           'mileage(km)',
                                                                                           'volume(cm3)']),
                                                                                         ('cat_1',
                                                                                          OneHotEncoder(sparse_output=False),
                                                                                          ['fuel',
                                                                                           'transmiss...
                                                                                                                                    'data_type': dtype('O'),
                                                                                                                                    'mapping': for parts       0
with damage     1
with mileage    2
dtype: int64}])),
                                                                                                          ('scaler',
                                                                                                           StandardScaler())]),
                                                                                          ['condition'])]))])),
                                       ('knn', KNeighborsRegressor())]),
             n_jobs=-1,
             param_grid=[{'knn': [KNeighborsRegressor()],
                          'knn__n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                          'knn__p': array([1, 2, 3, 4])}],
             scoring='neg_mean_absolute_percentage_error')

In [20]:
print("Наилучшие параметры:\n{}\n".format(result.best_params_))
print(result.best_score_)
grid_result = pd.DataFrame(result.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

Наилучшие параметры:
{'knn': KNeighborsRegressor(), 'knn__n_neighbors': 5, 'knn__p': 1}

-0.2444603699366911


,16,12,20,8,24,13,28,9,17,32,...,43,47,50,54,51,55,0,1,2,3
mean_fit_time,0.329095,0.333487,0.318173,0.366585,0.329675,0.360778,0.330343,0.379638,0.348478,0.360385,...,0.307671,0.310176,0.340476,0.359994,0.366233,0.311468,0.445606,0.411766,0.360092,0.334674
std_fit_time,0.014762,0.023261,0.018849,0.020529,0.018911,0.03504,0.021042,0.019968,0.015595,0.037442,...,0.019646,0.02923,0.052132,0.03819,0.063923,0.019744,0.027916,0.039114,0.028756,0.022858
mean_score_time,0.842588,0.757475,0.901605,0.683209,1.080196,0.786279,1.114143,0.657046,0.803482,1.294866,...,2.917239,3.114385,3.233989,3.31106,3.255574,2.82379,0.411096,0.362111,0.971226,0.995127
std_score_time,0.14918,0.131202,0.14814,0.09093,0.250022,0.039045,0.135156,0.053208,0.049354,0.22223,...,0.203856,0.181586,0.117821,0.078886,0.135437,0.297604,0.043452,0.061347,0.040495,0.03765
param_knn,KNeighborsRegressor(),KNeighborsRegressor(),KNeighborsRegressor(),KNeighborsRegressor(),KNeighborsRegressor(),KNeighborsRegressor(),KNeighborsRegressor(),KNeighborsRegressor(),KNeighborsRegressor(),KNeighborsRegressor(),...,KNeighborsRegressor(),KNeighborsRegressor(),KNeighborsRegressor(),KNeighborsRegressor(),KNeighborsRegressor(),KNeighborsRegressor(),KNeighborsRegressor(),KNeighborsRegressor(),KNeighborsRegressor(),KNeighborsRegressor()
param_knn__n_neighbors,5,4,6,3,7,4,8,3,5,9,...,11,12,13,14,13,14,1,1,1,1
param_knn__p,1,1,1,1,1,2,1,2,2,1,...,4,4,3,3,4,4,1,2,3,4
params,"{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo...",...,"{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo...","{'knn': KNeighborsRegressor(), 'knn__n_neighbo..."
split0_test_score,-0.239693,-0.238306,-0.243361,-0.242972,-0.245656,-0.248649,-0.250792,-0.254367,-0.249129,-0.252852,...,-0.268287,-0.268455,-0.26628,-0.268476,-0.26997,-0.271191,-0.271684,-0.276893,-0.274782,-0.272308
split1_test_score,-0.266444,-0.264602,-0.269397,-0.264224,-0.270161,-0.267723,-0.276176,-0.26528,-0.266152,-0.279349,...,-0.289859,-0.292586,-0.294724,-0.297079,-0.295071,-0.297883,-0.295669,-0.296158,-0.300449,-0.300117


In [32]:
knn_pipline = Pipeline(
    steps=[('preprocessor', preprocessor),
           ('knn', KNeighborsRegressor(n_neighbors=5, p=1))])

knn_pipline.fit(X_train, y_train)
y_pred = knn_pipline.predict(X_test)
print(mean_absolute_percentage_error(y_test, y_pred))

0.24252719441029919


In [22]:
lin_pipline = Pipeline(
    steps=[('preprocessor', preprocessor),
           ('lin', LinearRegression())])

lin_pipline.fit(X_train, y_train)
y_pred = lin_pipline.predict(X_test)
print(mean_absolute_percentage_error(y_test, y_pred))

0.8770366612985975


In [23]:
ridge_pipline = Pipeline(
    steps=[('preprocessor', preprocessor),
           ('regressor', Ridge())])

param_grid =[
    {'regressor': [Ridge()],
         'regressor__alpha': [1, 10, 30, 50, 100]
     }]


result = GridSearchCV(ridge_pipline, param_grid, cv=10, scoring = 'neg_mean_absolute_percentage_error') 
result.fit(X_train, y_train)

print("Наилучшие параметры:\n{}\n".format(result.best_params_))
print(result.best_score_)
grid_result = pd.DataFrame(result.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

Наилучшие параметры:
{'regressor': Ridge(), 'regressor__alpha': 100}

-0.903957324893692


,4,3,2,1,0
mean_fit_time,0.127464,0.125636,0.130898,0.124684,0.135764
std_fit_time,0.006148,0.010642,0.009883,0.007686,0.008961
mean_score_time,0.026104,0.024805,0.027205,0.026755,0.027407
std_score_time,0.001578,0.001601,0.002272,0.002481,0.002835
param_regressor,Ridge(),Ridge(),Ridge(),Ridge(),Ridge()
param_regressor__alpha,100,50,30,10,1
params,"{'regressor': Ridge(), 'regressor__alpha': 100}","{'regressor': Ridge(), 'regressor__alpha': 50}","{'regressor': Ridge(), 'regressor__alpha': 30}","{'regressor': Ridge(), 'regressor__alpha': 10}","{'regressor': Ridge(), 'regressor__alpha': 1}"
split0_test_score,-0.913515,-0.915745,-0.916643,-0.917542,-0.917949
split1_test_score,-0.846832,-0.849183,-0.850128,-0.851077,-0.851506
split2_test_score,-0.830018,-0.832116,-0.832963,-0.833812,-0.834197


In [24]:
lasso_pipline = Pipeline(
    steps=[('preprocessor', preprocessor),
           ('regressor', Lasso())])

param_grid =[
    {'regressor': [Lasso(max_iter=5000)],
         'regressor__alpha': [0.01, 0.05, 0.1, 0.5, 1]
     }]


result = GridSearchCV(lasso_pipline, param_grid, cv=10, scoring = 'neg_mean_absolute_percentage_error') 
result.fit(X_train, y_train)

print("Наилучшие параметры:\n{}\n".format(result.best_params_))
print(result.best_score_)
grid_result = pd.DataFrame(result.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

Наилучшие параметры:
{'regressor': Lasso(max_iter=5000), 'regressor__alpha': 1}

-0.9082529559123251


,4,3,2,1,0
mean_fit_time,0.134885,0.132635,0.133032,0.126029,0.136646
std_fit_time,0.007848,0.00705,0.006468,0.006959,0.004209
mean_score_time,0.026801,0.026303,0.027007,0.027106,0.026711
std_score_time,0.001777,0.002933,0.001416,0.002846,0.001417
param_regressor,Lasso(max_iter=5000),Lasso(max_iter=5000),Lasso(max_iter=5000),Lasso(max_iter=5000),Lasso(max_iter=5000)
param_regressor__alpha,1,0.5,0.1,0.05,0.01
params,"{'regressor': Lasso(max_iter=5000), 'regressor...","{'regressor': Lasso(max_iter=5000), 'regressor...","{'regressor': Lasso(max_iter=5000), 'regressor...","{'regressor': Lasso(max_iter=5000), 'regressor...","{'regressor': Lasso(max_iter=5000), 'regressor..."
split0_test_score,-0.917479,-0.917735,-0.917942,-0.917968,-0.917989
split1_test_score,-0.851272,-0.851411,-0.851524,-0.851539,-0.85155
split2_test_score,-0.833945,-0.834091,-0.83421,-0.834225,-0.834237


In [25]:
# Пока что модель knn показала себя лучше всех

In [26]:
rf_pipline = Pipeline(
    steps=[('preprocessor', preprocessor),
           ('regressor', RandomForestRegressor())])

param_grid =[
    {'regressor': [RandomForestRegressor()],
         'regressor__n_estimators': [350,500],
         'regressor__max_features': [None],
         'regressor__min_samples_leaf': [1,2,3,5,10,15]
     }]


grid = GridSearchCV(rf_pipline,
                       param_grid,
                       scoring='neg_mean_absolute_percentage_error',
                       cv=10,
                       n_jobs=-1)

grid.fit(X_train, y_train)
grid 

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        Pipeline(steps=[('data_transformer',
                                                         ColumnTransformer(transformers=[('numerical',
                                                                                          Pipeline(steps=[('missing_num',
                                                                                                           IterativeImputer(max_iter=20,
                                                                                                                            missing_values=0)),
                                                                                                          ('scaler',
                                                                                                           StandardScaler())]),
                                                                                          ['age',
                                                                                           'mileage(km)',
                                                                                           'volume(cm3)']),
                                                                                         ('cat_1',
                                                                                          OneHotEncoder(sparse_output=False),
                                                                                          ['fuel',
                                                                                           'transmiss...
with damage     1
with mileage    2
dtype: int64}])),
                                                                                                          ('scaler',
                                                                                                           StandardScaler())]),
                                                                                          ['condition'])]))])),
                                       ('regressor', RandomForestRegressor())]),
             n_jobs=-1,
             param_grid=[{'regressor': [RandomForestRegressor()],
                          'regressor__max_features': [None],
                          'regressor__min_samples_leaf': [1, 2, 3, 5, 10, 15],
                          'regressor__n_estimators': [350, 500]}],
             scoring='neg_mean_absolute_percentage_error')

In [27]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

,3,2,5,4,1,0,6,7,9,8,11,10
mean_fit_time,91.945812,68.230707,83.632286,57.430417,127.393172,88.104286,48.573169,69.690388,58.196113,39.145129,47.060362,34.752376
std_fit_time,1.660679,1.766791,0.776405,0.791691,1.947209,0.890544,1.27831,0.817972,1.766098,0.367418,4.225614,0.307523
mean_score_time,0.880929,0.82569,0.717284,0.470781,7.718736,5.056736,0.359745,0.531942,0.400863,0.323637,0.35479,0.269782
std_score_time,0.236173,0.432724,0.147624,0.037665,5.312811,1.658485,0.054889,0.085795,0.035063,0.077123,0.094073,0.021887
param_regressor,RandomForestRegressor(),RandomForestRegressor(),RandomForestRegressor(),RandomForestRegressor(),RandomForestRegressor(),RandomForestRegressor(),RandomForestRegressor(),RandomForestRegressor(),RandomForestRegressor(),RandomForestRegressor(),RandomForestRegressor(),RandomForestRegressor()
param_regressor__max_features,None,None,None,None,None,None,None,None,None,None,None,None
param_regressor__min_samples_leaf,2,2,3,3,1,1,5,5,10,10,15,15
param_regressor__n_estimators,500,350,500,350,500,350,350,500,500,350,500,350
params,"{'regressor': RandomForestRegressor(), 'regres...","{'regressor': RandomForestRegressor(), 'regres...","{'regressor': RandomForestRegressor(), 'regres...","{'regressor': RandomForestRegressor(), 'regres...","{'regressor': RandomForestRegressor(), 'regres...","{'regressor': RandomForestRegressor(), 'regres...","{'regressor': RandomForestRegressor(), 'regres...","{'regressor': RandomForestRegressor(), 'regres...","{'regressor': RandomForestRegressor(), 'regres...","{'regressor': RandomForestRegressor(), 'regres...","{'regressor': RandomForestRegressor(), 'regres...","{'regressor': RandomForestRegressor(), 'regres..."
split0_test_score,-0.230351,-0.230638,-0.229733,-0.231072,-0.235078,-0.234997,-0.23482,-0.2341,-0.243597,-0.243776,-0.253891,-0.254303


In [39]:
rf_pipline = Pipeline(
    steps=[('preprocessor', preprocessor),
           ('regressor', RandomForestRegressor(n_estimators=500 , min_samples_leaf=2, random_state=42))])
rf_pipline.fit(X_train,y_train)

y_pred = rf_pipline.predict(X_test)
print(mean_absolute_percentage_error(y_test, y_pred))

0.23093902687595022


In [34]:
rf_pipline = Pipeline(
    steps=[('preprocessor', preprocessor),
           ('regressor', RandomForestRegressor(n_estimators=350 , min_samples_leaf=2, random_state=42))])
rf_pipline.fit(X_train,y_train)

y_pred = rf_pipline.predict(X_test)
print(mean_absolute_percentage_error(y_test, y_pred))

0.23095809072537238


In [38]:
rf_pipline = Pipeline(
    steps=[('preprocessor', preprocessor),
           ('regressor', RandomForestRegressor(n_estimators=500, min_samples_leaf=3, random_state=42))])
rf_pipline.fit(X_train,y_train)

y_pred = rf_pipline.predict(X_test)
print(mean_absolute_percentage_error(y_test, y_pred))

0.23122430775954256


In [ ]:
# Модель rf лучше остальных. Выбираем ее